# Row 데이터 불필요한 데이터 제거해서 크기 줄이기.

같은 폴더 안에 있는 data_refine.py와 코드는 같다. 파이썬 파일를 노트북에서 결과 볼라고 가져온거

In [2]:
import pyspark
import pyspark.sql
import sys, os, re, iso8601, datetime
import timeit

conf = pyspark.SparkConf().setAll([('spark.driver.memory', '2g')])
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc).builder.getOrCreate()


project_home = '..'

In [3]:
# 데이터 읽어오기
raw_dataframe = spark.read.parquet("{}/data/Raw_Data.parquet".format(project_home))

# 테이블 등록
raw_dataframe.registerTempTable("rawdata_table")

In [23]:
# raw 데이터 개수
raw_dataframe.count()

5819079

In [4]:
# null값을 카운트 한 다음, 퍼샌트로 만들어줌 (where 대신 filter 도 가능)
def column_nullCount_ToParcent(dataframe,column):
    return round((dataframe.where(dataframe[column].isNull()).count()/dataframe.count()) * 100,2)

# 각 컬럼 별로 null 값 검사(%)
null_parcent = [(column, column_nullCount_ToParcent(raw_dataframe,column)) for column in raw_dataframe.columns]
null_parcent

[('Year', 0.0),
 ('Quarter', 0.0),
 ('Month', 0.0),
 ('DayofMonth', 0.0),
 ('DayOfWeek', 0.0),
 ('FlightDate', 0.0),
 ('Carrier', 0.0),
 ('TailNum', 0.25),
 ('FlightNum', 0.0),
 ('Origin', 0.0),
 ('OriginCityName', 0.0),
 ('OriginState', 0.0),
 ('Dest', 0.0),
 ('DestCityName', 0.0),
 ('DestState', 0.0),
 ('DepTime', 1.48),
 ('DepDelay', 1.48),
 ('DepDelayMinutes', 1.48),
 ('TaxiOut', 1.53),
 ('TaxiIn', 1.59),
 ('WheelsOff', 1.53),
 ('WheelsOn', 1.59),
 ('ArrTime', 1.59),
 ('ArrDelay', 1.81),
 ('ArrDelayMinutes', 1.81),
 ('Cancelled', 0.0),
 ('Diverted', 0.0),
 ('ActualElapsedTime', 1.81),
 ('AirTime', 1.81),
 ('Flights', 0.0),
 ('Distance', 0.0),
 ('CarrierDelay', 81.72),
 ('WeatherDelay', 81.72),
 ('NASDelay', 81.72),
 ('SecurityDelay', 81.72),
 ('LateAircraftDelay', 81.72),
 ('CRSDepTime', 0.0),
 ('CRSArrTime', 0.0)]

In [5]:
# null 비율이 10%이하인 컬럼 추출
null_10parcent_down_col = []

for column, nullParcent in null_parcent:
    if nullParcent < 10:
        null_10parcent_down_col.append(column)
        
null_10parcent_down_col

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Carrier',
 'TailNum',
 'FlightNum',
 'Origin',
 'OriginCityName',
 'OriginState',
 'Dest',
 'DestCityName',
 'DestState',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'TaxiOut',
 'TaxiIn',
 'WheelsOff',
 'WheelsOn',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'Cancelled',
 'Diverted',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'CRSDepTime',
 'CRSArrTime']

In [6]:
# 스파크SQL을 이용하여 null값이 10%이하인 컬럼만 선택
fit_dataframe = spark.sql(
  """SELECT {} FROM rawdata_table
  """.format(','.join(null_10parcent_down_col))
)

fit_dataframe.columns


['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Carrier',
 'TailNum',
 'FlightNum',
 'Origin',
 'OriginCityName',
 'OriginState',
 'Dest',
 'DestCityName',
 'DestState',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'TaxiOut',
 'TaxiIn',
 'WheelsOff',
 'WheelsOn',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'Cancelled',
 'Diverted',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'CRSDepTime',
 'CRSArrTime']

In [7]:
# 나머지 null인 행들 모두 제거 
print("지우기 전:",fit_dataframe.count())
fit_notnull_dataframe = fit_dataframe.na.drop()
print("지운 후:",fit_notnull_dataframe.count())


지우기 전: 5819079
지운 후: 5714008


In [8]:
# 정상적으로 제거 되었는지 확인
null_parcent = [(column, column_nullCount_ToParcent(fit_notnull_dataframe,column)) for column in fit_notnull_dataframe.columns]
null_parcent

[('Year', 0.0),
 ('Quarter', 0.0),
 ('Month', 0.0),
 ('DayofMonth', 0.0),
 ('DayOfWeek', 0.0),
 ('FlightDate', 0.0),
 ('Carrier', 0.0),
 ('TailNum', 0.0),
 ('FlightNum', 0.0),
 ('Origin', 0.0),
 ('OriginCityName', 0.0),
 ('OriginState', 0.0),
 ('Dest', 0.0),
 ('DestCityName', 0.0),
 ('DestState', 0.0),
 ('DepTime', 0.0),
 ('DepDelay', 0.0),
 ('DepDelayMinutes', 0.0),
 ('TaxiOut', 0.0),
 ('TaxiIn', 0.0),
 ('WheelsOff', 0.0),
 ('WheelsOn', 0.0),
 ('ArrTime', 0.0),
 ('ArrDelay', 0.0),
 ('ArrDelayMinutes', 0.0),
 ('Cancelled', 0.0),
 ('Diverted', 0.0),
 ('ActualElapsedTime', 0.0),
 ('AirTime', 0.0),
 ('Flights', 0.0),
 ('Distance', 0.0),
 ('CRSDepTime', 0.0),
 ('CRSArrTime', 0.0)]

In [10]:
# 파퀘이로 저장
fit_notnull_dataframe.repartition(1).write.mode('overwrite').parquet("{}/data/Refined_Data.parquet".format(project_home))

